In [3]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde molecule in a cc-pVDZ basis set.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cs_cqed_cis import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [6]:

# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c2v
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-10,
               'd_convergence' : 1e-10}

psi4.set_options(options_dict)
mol = psi4.geometry(molstr)


energy, wfn = psi4.energy("scf/cc-pVDZ", molecule=mol, return_wfn=True)
print(energy)

HOMO = 8
LUMO = 9 # These are is the HOMO and LUMO for HF.
                # The HOMO/LUMO is based on the total number of electrons. HF has 10 electrons
                # and each orbital can have two electrons. 5 is the HOMO and 6 is the LUMO



psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS', 'DENSITY'], # calculate orbitals
    'CUBEPROP_ORBITALS': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],    # calculate HOMO and LUMO. 
})
cube = psi4.cubeprop(wfn)

-113.87722217661772


In [7]:
import fortecubeview
fortecubeview.plot()

CubeViewer: listing cube files from the directory .
Reading 19 cube files


/opt/anaconda3/lib//python3.8/site-packages/fortecubeview/py3js_renderer.py:1096: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level * 0.995)
/opt/anaconda3/lib//python3.8/site-packages/fortecubeview/cube_file.py:213: RuntimeWarning: invalid value encountered in double_scalars
  if partial_sum / sum < fraction:


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('Density (alpha)', 'Density (beta)', 'Density…

In [ ]:
# different magnitudes of the lambda vector
l = 0.5

no_lam = np.array([0., 0., 0.])
lam_y = np.array([0, l, 0])
lam_z = np.array([0, 0, l])
lam_yz = np.array([0, l/np.sqrt(2), l/np.sqrt(2)])
    
# results for y-polarized
y_dict = cqed_rhf(lam_y, molstr, options_dict)
    
z_dict = cqed_rhf(lam_z, molstr, options_dict)

yz_dict = cqed_rhf(lam_yz, molstr, options_dict)

no_lam_dict = cqed_rhf(no_lam, molstr, options_dict)



In [ ]:
# RHF Wavefunction dict from the original RHF wavefunction object 'wfn'
rhf_wfn_dict = psi4.core.Wavefunction.to_file(wfn)

D_psi = rhf_wfn_dict['matrix']['Da']

D_my = yz_dict["CQED-RHF DENSITY MATRIX"]

C_my = yz_dict["CQED-RHF C"]

print(D_my - D_psi)


In [ ]:
rhf_wfn_dict['matrix']['Da'] = D_my
rhf_wfn_dict['matrix']['Db'] = D_psi

rhf_wfn_dict['matrix']['Ca'] = C_my
rhf_wfn_dict['matrix']['Cb'] = C_my
# Now create a new wavefunction object that has the CQED-RHF orbitals 'cqedrhf_wfn'
cqedrhf_wfn = psi4.core.Wavefunction.from_file(rhf_wfn_dict) 
# Confirm you have copied the CQED-RHF orbitals properly 
# by again getting a dic of the wavefunction and comparing
# the orbitals to the original numpy array 'C' that resulted
# from the CQED-RHF iterations
cqedrhf_wfn_dict = psi4.core.Wavefunction.to_file(cqedrhf_wfn)

assert np.allclose(cqedrhf_wfn_dict['matrix']['Db'], D_my, 1e-6)
#assert np.allclose(cqedrhf_wfn_dict['matrix']['Cb'], C, 1e-6)



In [ ]:

HOMO = 8
LUMO = 9 # These are is the HOMO and LUMO for HF.
                # The HOMO/LUMO is based on the total number of electrons. HF has 10 electrons
                # and each orbital can have two electrons. 5 is the HOMO and 6 is the LUMO


psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS', 'DENSITY'], # calculate orbitals
    'CUBEPROP_ORBITALS': [HOMO-1, HOMO, LUMO, LUMO+1],    # calculate HOMO and LUMO. 
})
cube = psi4.cubeprop(cqedrhf_wfn)

In [ ]:
import fortecubeview
fortecubeview.plot()

In [ ]:
mol.save_xyz_file("geom.xyz", 1)

In [ ]:
f = open("geom.xyz")
xyz = f.read()
f.close()

Run the scan below to generate data for Figure 1: (Top)  Total  CQED-RHF  energy,  (middle)  canonical  RHFcontribution to the total energy, (bottom) Pauli-Fierz contribution tothe total energy as a function of|λ|along they−and/orz−axes

In [ ]:
import py3Dmol

In [ ]:
def open_cube(cube_file):
    """
    Open and parse a cube file.
    
    The low and high isovalues are returned, along with the cube values. The low and high values represent the
    isocontour range for 85% of the density.
    
    Parameters
    ----------
    cube_file : str
        The filepath to the cube file to read.
    
    Returns
    -------
    low : float
        Low limit for isocontour range to capture 85% of the density.
    high : float
        High limit for isocontour range to capture 85% of the density.
    
    """
    
    # Get isovalues
    f = open(cube_file, "r")
    f.readline()
    line = f.readline()
    f.close()
    split_line = line.split(":")
    
    isovalues = split_line[-1][2:-2]
    isovalues = isovalues.split(",")
    
    high = float(isovalues[1])
    low = float(isovalues[0])
    
    # Read cube file
    f = open(cube_file, "r")
    data = f.read()
    f.close()
    print(low, high)
    return low, high, data

In [ ]:
import glob


In [ ]:
cube_files = glob.glob("*.cube")
print(cube_files)

cube = cube_files[3]
isohigh, isolow, orbital = open_cube(cube)
    
# Process the file name to get the orbital
orbital_num = cube.split("_")[2]
    
print(F"Showing visualization for orbital {orbital_num}")
    
# Use pymol3D to make visualization
v = py3Dmol.view()
    
# Add surfaces
v.addVolumetricData(orbital, "cube", {'isoval': isohigh, 'color': "red", 'opacity': 0.90})
#v.addVolumetricData(orbital, "cube", {'isoval': isolow, 'color': "blue", 'opacity': 0.90})
    
# Add molecule
v.addModel(xyz, 'mol')
v.setStyle({'stick':{}})
    
# Zoom to molecule and show
v.zoomTo()
v.show()